In [ ]:
# Below the comment in the upper right corner, click on the down arrow (triangle) for more connection options -> Change runtime type -> T4 GPU

In [ ]:
# colab notes:
# 1. use ! for most linux commands, and % for changing directory
# 2. conda environment cannot be kept after using 'source activate xxx'

#!git clone https://github.com/ku21fan/COO-Comic-Onomatopoeia.git
!git clone https://github.com/WeNYLee/COO-Comic-Onomatopoeia.git

# install conda
!pip install -q condacolab
import condacolab
condacolab.install()

%cd /content/COO-Comic-Onomatopoeia

# create an environment with python 3.8
# -y is to automatically agree some requests while creating the environment
!conda create -n COO python=3.8 -y

# install torch and torchvision
!source activate COO && python -m pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install for ABCNetv2
!source activate COO && python -m pip install detectron2==0.5 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
!source activate COO && python -m pip install opencv-python
!source activate COO && python -m pip install scikit-image==0.19.3
!source activate COO && python -m pip install pillow==8.4.0

# install for TRBA
!source activate COO && python -m pip install -r TRBA/requirements.txt

Cloning into 'COO-Comic-Onomatopoeia'...
remote: Enumerating objects: 1936, done.
remote: Counting objects: 100% (492/492), done.
remote: Compressing objects: 100% (290/290), done.
remote: Total 1936 (delta 239), reused 437 (delta 190), pack-reused 1444
Receiving objects: 100% (1936/1936), 33.10 MiB | 19.19 MiB/s, done.
Resolving deltas: 100% (439/439), done.
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:10
🔁 Restarting kernel...
/content/COO-Comic-Onomatopoeia
Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/COO

  added / updated specs:
    - pytho

In [ ]:
%cd /content/COO-Comic-Onomatopoeia/ABCNetv2/
!source activate COO && python setup.py build develop

/content/COO-Comic-Onomatopoeia/ABCNetv2
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.8
creating build/lib.linux-x86_64-3.8/adet
copying adet/__init__.py -> build/lib.linux-x86_64-3.8/adet
creating build/lib.linux-x86_64-3.8/adet/data
copying adet/data/fcpose_dataset_mapper.py -> build/lib.linux-x86_64-3.8/adet/data
copying adet/data/augmentation.py -> build/lib.linux-x86_64-3.8/adet/data
copying adet/data/builtin.py -> build/lib.linux-x86_64-3.8/adet/data
copying adet/data/dataset_mapper.py -> build/lib.linux-x86_64-3.8/adet/data
copying adet/data/detection_utils.py -> build/lib.linux-x86_64-3.8/adet/data
copying adet/data/__init__.py -> build/lib.linux-x86_64-3.8/adet/data
creating build/lib.linux-x86_64-3.8/adet/layers
copying adet/layers/bezier_align.py -> build/lib.linux-x86_64-3.8/adet/layers
copying adet/layers/gcn.py -> build/lib.linux-x86_64-3.8/adet/layers
copying adet/layers/ml_nms.py -> build/lib.linux-x86_64-3.8/adet/layers
copying adet/l

In [ ]:
# download a pretrained ABCNetv2 model
!wget https://www.dropbox.com/s/cg63gsnf7741vjw/ABCNetv2.pth

--2024-05-15 14:26:24--  https://www.dropbox.com/s/cg63gsnf7741vjw/ABCNetv2.pth
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/cg63gsnf7741vjw/ABCNetv2.pth [following]
--2024-05-15 14:26:24--  https://www.dropbox.com/s/raw/cg63gsnf7741vjw/ABCNetv2.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1f8c999c9ce39888af3ac57b81.dl.dropboxusercontent.com/cd/0/inline/CS_IU6c_fszCm3HLcLPXBbovpyfL___dqUt7KkKZT-worAXPmsySx-ZN_SOSgPdLh1_vFEM3QS0Ji5FCoSj9UQYV3ds17C3Z2XrECQm1wJc7AjX1nBEJlyDKQDzPiQE-2YujF3ounsk-hX0HNSxo0_lt/file# [following]
--2024-05-15 14:26:25--  https://uc1f8c999c9ce39888af3ac57b81.dl.dropboxusercontent.com/cd/0/inline/CS_IU6c_fszCm3HLcLPXBbovpyfL___dqUt7KkKZT-worAXPmsySx-ZN_SOSgPdLh1_vFEM3QS0Ji5FCoSj9UQYV3ds17C3Z2XrECQ

In [ ]:
# create folders for input images and output images
!mkdir -p demo_images/test
!mkdir -p demo_results/test

In [ ]:
# mount google drive into google colab to obtain test images
from google.colab import drive
import os

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/dataset/* demo_images/test/

In [ ]:
# run ABCNet v2
!source activate COO && CUDA_VISIBLE_DEVICES=0 python demo/demo.py --config-file configs/eval.yaml --input demo_images/test/ --output demo_results/test/ --opts MODEL.WEIGHTS ABCNetv2.pth

[05/15 14:32:27 detectron2]: Arguments: Namespace(confidence_threshold=0.3, config_file='configs/eval.yaml', input=['demo_images/test/'], opts=['MODEL.WEIGHTS', 'ABCNetv2.pth'], output='demo_results/test/', video_input=None, webcam=False)
WARNING [05/15 14:32:27 d2.config.compat]: Config 'configs/eval.yaml' has no VERSION. Assuming it to be compatible with latest v2.
  0% 0/11 [00:00<?, ?it/s]/usr/local/envs/COO/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/usr/local/envs/COO/lib/python3.8/site-packages/to

In [ ]:
!mkdir -p demo_results/test_cropped
!mv demo_results/test/*cropped*.jpg demo_results/test_cropped/

In [ ]:
%cd /content/COO-Comic-Onomatopoeia/TRBA

/content/COO-Comic-Onomatopoeia/TRBA


In [ ]:
# download a pretrained TRBA model
!wget https://www.dropbox.com/sh/lx61z7gq5yzkp02/AAC8ZPGIdYmqAxihrvVk6o26a/TRBA_Rot%2BSAR%2BHardROIhalf%2B2D.pth

--2024-05-15 14:38:18--  https://www.dropbox.com/sh/lx61z7gq5yzkp02/AAC8ZPGIdYmqAxihrvVk6o26a/TRBA_Rot%2BSAR%2BHardROIhalf%2B2D.pth
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /sh/raw/lx61z7gq5yzkp02/AAC8ZPGIdYmqAxihrvVk6o26a/TRBA_Rot%2BSAR%2BHardROIhalf%2B2D.pth [following]
--2024-05-15 14:38:19--  https://www.dropbox.com/sh/raw/lx61z7gq5yzkp02/AAC8ZPGIdYmqAxihrvVk6o26a/TRBA_Rot%2BSAR%2BHardROIhalf%2B2D.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucde77139c732570103ff2c37749.dl.dropboxusercontent.com/cd/0/inline/CS-ZG3mgVTi1TfE5Pk4NtKF-e6id-KXBFiiksLV4M5bR7F_ffLRTwrOgIf3LxRFZpyZTAgiHncRTo_pUYmnPyMATfWSRVMXgdYq5aZKat3w23cTjuiDUge3GNPvh4W_WLBVIXYxjpRK_bvnBqGeconHL/file# [following]
--2024-05-15 14:38:19--  https://ucde77139c732

In [ ]:
# run TRBA
!source activate COO && CUDA_VISIBLE_DEVICES=0 python3 demo.py --model_name TRBA --SARdecode --twoD --imgH 100 --image_folder /content/COO-Comic-Onomatopoeia/ABCNetv2/demo_results/test_cropped/ --saved_model TRBA_Rot+SAR+HardROIhalf+2D.pth

# of tokens and characters: 187
model input parameters 100 100 20 3 512 256 187 25 TPS ResNet BiLSTM Attn
loading pretrained model from TRBA_Rot+SAR+HardROIhalf+2D.pth
/usr/local/envs/COO/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/envs/COO/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_p

In [ ]:
%cd /content/COO-Comic-Onomatopoeia/ABCNetv2/

/content/COO-Comic-Onomatopoeia/ABCNetv2


In [ ]:
!cp -r demo_results /content/gdrive/My\ Drive/Colab\ Notebooks/